# Compare datasets

June 11, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob

from scipy import fftpack
from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

[NbConvertApp] WARNING | pattern 'modules_image_analysis.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook outp

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Compare results

In [5]:
dataset1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/very_large_dataset_train.npy'
dataset2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/peter_dataset/raw_train.npy'
dataset3='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_2_smooothing_200k/train.npy'
# dataset4='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_2_smooothing_200k/val.npy'
dataset4='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_1_no_smoothing_200k/train.npy'

dataset_list=[]
for i in [dataset2,dataset3,dataset4]: dataset_list.append(i)
print(dataset_list)
                     
s_input=[[],[],[]]
for count,fname in enumerate(dataset_list):
    print(fname)
    arr=np.load(fname)
    num_samples=arr.shape[0]
    print(arr.shape)
    idxs=np.random.choice(np.arange(num_samples),size=50000,replace=False)
#     idxs=np.arange(1000,51000)
    arr=arr[idxs]
#     s_input[count]=arr[:,:,:,0]
    s_input[count]=f_transform(arr[:,:,:,0])



['/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/peter_dataset/raw_train.npy', '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_2_smooothing_200k/train.npy', '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_1_no_smoothing_200k/train.npy']
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/peter_dataset/raw_train.npy
(197000, 128, 128, 1)
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_2_smooothing_200k/train.npy
(200000, 128, 128, 1)
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_1_no_smoothing_200k/train.npy
(200000, 128, 128, 1)


In [6]:

def f_widget_compare(sample_names,sample_dict,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
    
    ### Crop out large pixel values
    for key in sample_names:
        print(sample_dict[key].shape)
        sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
        print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
        
    hist_range=(0,0.996)
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        hist_range=(0,2000)

    
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,log_scale=log_scale)



In [7]:
dict_samples={'peter_data': s_input[0], 'smoothing':s_input[1],'no_smoothing':s_input[2]}

interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),
                bins=SelectionSlider(options=np.arange(10,200,10),value=50),
                mode=['avg','simple'])

interactive(children=(SelectMultiple(description='sample_names', options=('peter_data', 'smoothing', 'no_smoot…

<function __main__.f_widget_compare(sample_names, sample_dict, Fig_type='pixel', rescale=True, log_scale=True, bins=25, mode='avg', normalize=True)>